### Part I: Demand Unconstraining: Physical products with limited inventory
##### a. Double Exponential Smoothing (DES) estimations

In [9]:
import pandas as pd
df_grocery = pd.read_excel("grocery_data.xlsx")
df_grocery.info()

# remove first row and col and last column
df_grocery = df_grocery.iloc[1:,1:17]
df_grocery

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       21 non-null     object 
 1   Hourly interval  21 non-null     int64  
 2   Unnamed: 2       21 non-null     int64  
 3   Unnamed: 3       21 non-null     int64  
 4   Unnamed: 4       21 non-null     int64  
 5   Unnamed: 5       21 non-null     int64  
 6   Unnamed: 6       21 non-null     int64  
 7   Unnamed: 7       21 non-null     int64  
 8   Unnamed: 8       21 non-null     int64  
 9   Unnamed: 9       20 non-null     float64
 10  Unnamed: 10      20 non-null     float64
 11  Unnamed: 11      17 non-null     float64
 12  Unnamed: 12      17 non-null     float64
 13  Unnamed: 13      15 non-null     float64
 14  Unnamed: 14      15 non-null     float64
 15  Unnamed: 15      11 non-null     float64
 16  Unnamed: 16      8 non-null      float64
 17  Total            2

,Hourly interval,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
1,2,4,4,2,4,2,4,1,5.0,4.0,5.0,3.0,NaN,NaN,NaN,NaN
2,3,4,4,5,4,5,4,2,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1,3,2,2,3,6,4,2,3.0,3.0,3.0,4.0,2.0,2.0,NaN,NaN
4,2,3,4,1,4,3,3,1,3.0,3.0,4.0,2.0,3.0,2.0,2.0,NaN
5,2,5,5,3,5,5,5,2,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
6,1,3,3,1,3,3,2,1,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0
7,3,4,3,2,4,5,5,3,2.0,4.0,3.0,2.0,NaN,NaN,NaN,NaN
8,1,2,2,1,1,1,2,1,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0
9,1,2,3,2,2,2,2,1,2.0,2.0,2.0,2.0,2.0,3.0,2.0,1.0
10,2,6,6,3,4,2,4,2,6.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# rename columns
import numpy as np
df_grocery.columns = np.arange(1,17)
df_grocery.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
1,2,4,4,2,4,2,4,1,5.0,4.0,5.0,3.0,NaN,NaN,NaN,NaN
2,3,4,4,5,4,5,4,2,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1,3,2,2,3,6,4,2,3.0,3.0,3.0,4.0,2.0,2.0,NaN,NaN
4,2,3,4,1,4,3,3,1,3.0,3.0,4.0,2.0,3.0,2.0,2.0,NaN
5,2,5,5,3,5,5,5,2,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Step 1: Tease out the constrained constrained booking curves
# adjust the grocery data of sales by removing the sales possibly affected by out-of-stock
# check at which hour the demand becomes censored
# note that most summarize methods could be applied row-wise with axis = 1
df_count= df_grocery.count(axis=1)
for i in range(1,21):
    if df_grocery.isnull().any(axis = 1).loc[i]: # with or without = True makes no difference
        # also works: df_grocery.iloc[:,-1].isnull()
        df_grocery.loc[i,df_count[i]] = np.nan
# verify that capturing hour the demand becomes censored
df_count

df_grocery

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
1,2,4,4,2,4,2,4,1.0,5.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN
2,3,4,4,5,4,5,4,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,3,2,2,3,6,4,2.0,3.0,3.0,3.0,4.0,2.0,NaN,NaN,NaN
4,2,3,4,1,4,3,3,1.0,3.0,3.0,4.0,2.0,3.0,2.0,NaN,NaN
5,2,5,5,3,5,5,5,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,3,3,1,3,3,2,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0
7,3,4,3,2,4,5,5,3.0,2.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN
8,1,2,2,1,1,1,2,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0
9,1,2,3,2,2,2,2,1.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,1.0
10,2,6,6,3,4,2,4,2.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# STEP 1: calculate hourly cumulative demand 
import numpy as np
df_cumsales = np.cumsum(df_grocery,axis=1)
df_cumsales

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
1,2.0,6.0,10.0,12.0,16.0,18.0,22.0,23.0,28.0,32.0,37.0,NaN,NaN,NaN,NaN,NaN
2,3.0,7.0,11.0,16.0,20.0,25.0,29.0,31.0,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,4.0,6.0,8.0,11.0,17.0,21.0,23.0,26.0,29.0,32.0,36.0,38.0,NaN,NaN,NaN
4,2.0,5.0,9.0,10.0,14.0,17.0,20.0,21.0,24.0,27.0,31.0,33.0,36.0,38.0,NaN,NaN
5,2.0,7.0,12.0,15.0,20.0,25.0,30.0,32.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.0,4.0,7.0,8.0,11.0,14.0,16.0,17.0,18.0,20.0,22.0,24.0,26.0,28.0,30.0,31.0
7,3.0,7.0,10.0,12.0,16.0,21.0,26.0,29.0,31.0,35.0,38.0,NaN,NaN,NaN,NaN,NaN
8,1.0,3.0,5.0,6.0,7.0,8.0,10.0,11.0,12.0,13.0,15.0,16.0,17.0,18.0,19.0,20.0
9,1.0,3.0,6.0,8.0,10.0,12.0,14.0,15.0,17.0,19.0,21.0,23.0,25.0,28.0,30.0,31.0
10,2.0,8.0,14.0,17.0,21.0,23.0,27.0,29.0,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# strategy:
# create a big loop that takes a censored curve as input and fills in the nan's 
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.api import ExponentialSmoothing
from itertools import product

# write a function that takes alpha,beta,series as input 
# and produces the traning sample MSE
# allow 3 periods for burn-in
def DESMSE(alpha,beta,series):
    return mean_squared_error(series[3:],
                              ExponentialSmoothing(series,trend="add", initialization_method
                                                   ='estimated').fit(smoothing_level = alpha,smoothing_trend
                                                                     = beta).predict(3,series.size-1))

# the function below allows you to construct a dataframe of alpha-beta combinations
def expand_grid(dictionary):
    return pd.DataFrame([row for row in product(*dictionary.values())], 
                       columns=dictionary.keys())
dictionary = {'alpha': np.arange(0.1, 1.0, 0.1), 
              'beta': np.arange(0.1, 1.0, 0.1)}

In [15]:
for row_number in np.arange(0,df_cumsales.shape[0]):
    
    temp = df_cumsales.iloc[row_number,:].dropna().reset_index(drop=True)
    temp = np.append(temp[0],temp[1:].values - temp[:-1].values)
    
    
    des_results = expand_grid(dictionary)
    
    
    des_results["mse"] = np.repeat(0,len(des_results))
    for alpha in np.arange(0.1, 1.1, 0.1):
        for beta in np.arange(0.1, 1.1, 0.1):
           des_results.loc[(des_results["alpha"] == alpha) 
                           & (des_results["beta"] == beta),"mse"] = DESMSE(alpha,beta,temp)
    
    # find the best (alpha beta): sort and put on row 1
    des_results = des_results.sort_values("mse")
    
    if (16-temp.size) > 0:
      df_grocery['censored'] = True
      predicted_daily_arrival = ExponentialSmoothing(temp,trend="add", initialization_method='estimated').fit(smoothing_level = des_results.iloc[0,0],smoothing_trend = des_results.iloc[0,1]).forecast(16-temp.size)
    
      df_cumsales.iloc[row_number,temp.size:] = predicted_daily_arrival.cumsum() + df_cumsales.iloc[row_number,temp.size-1]
    else:
      df_grocery['censored']= False 
      continue


In [16]:
df_grocery['des'] = df_cumsales[16].round()
df_grocery.head(20)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,censored,des
1,2,4,4,2,4,2,4,1.0,5.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN,False,60.0
2,3,4,4,5,4,5,4,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,60.0
3,1,3,2,2,3,6,4,2.0,3.0,3.0,3.0,4.0,2.0,NaN,NaN,NaN,False,49.0
4,2,3,4,1,4,3,3,1.0,3.0,3.0,4.0,2.0,3.0,2.0,NaN,NaN,False,43.0
5,2,5,5,3,5,5,5,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,65.0
6,1,3,3,1,3,3,2,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,False,31.0
7,3,4,3,2,4,5,5,3.0,2.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN,False,55.0
8,1,2,2,1,1,1,2,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,False,20.0
9,1,2,3,2,2,2,2,1.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,1.0,False,31.0
10,2,6,6,3,4,2,4,2.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,61.0


##### b. Averaging method estimations

In [34]:
#import pandas as pd
import numpy as np
# remove first row and col and last column
df_grocery = pd.read_excel("grocery_data.xlsx")
df_grocery = df_grocery.iloc[1:,1:17]
df_grocery.columns = np.arange(1,17)
df_grocery

df_count= df_grocery.count(axis=1)
for i in range(1,17):
    if df_grocery.isnull().any(axis = 1).loc[i]: # with or without = True makes no difference
        # also works: df_grocery.iloc[:,-1].isnull()
        df_grocery.loc[i,df_count[i]] = np.nan
# verify that capturing hour the demand becomes censored
df_count

df_grocery

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
1,2,4,4,2,4,2,4,1.0,5.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN
2,3,4,4,5,4,5,4,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,3,2,2,3,6,4,2.0,3.0,3.0,3.0,4.0,2.0,NaN,NaN,NaN
4,2,3,4,1,4,3,3,1.0,3.0,3.0,4.0,2.0,3.0,2.0,NaN,NaN
5,2,5,5,3,5,5,5,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,3,3,1,3,3,2,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0
7,3,4,3,2,4,5,5,3.0,2.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN
8,1,2,2,1,1,1,2,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0
9,1,2,3,2,2,2,2,1.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,1.0
10,2,6,6,3,4,2,4,2.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:

# convert dataframe from 'wide' to 'long'
df_avg = pd.melt(df_grocery.iloc[:,:].reset_index(),id_vars='index',var_name = 'hour',value_name='hourly_sales') 
# sort by curve (day) and hour 
df_avg = df_avg.sort_values(by=['index','hour'])
df_avg.head(55)

,index,hour,hourly_sales
0,1,1,2.0
20,1,2,4.0
40,1,3,4.0
60,1,4,2.0
80,1,5,4.0
100,1,6,2.0
120,1,7,4.0
140,1,8,1.0
160,1,9,5.0
180,1,10,4.0


In [36]:
df_avg['censor'] = df_avg['hourly_sales'].isnull() == True
# avg hourly booking across uncensored days
temp = df_avg.groupby('hour').hourly_sales.mean()
temp = temp.reset_index().rename(columns = {'index' : "hour", 'hourly_sales' : 'hourly_sales_avg'})

temp['hour'] = temp.hour.astype('object')
df_avg = pd.merge(df_avg,temp,how='left',on='hour')
# hourly_sales_original captures the last hour sales before censoring
df_grocery_original = pd.read_excel("grocery_data.xlsx").iloc[1:,1:17]
df_grocery_original.columns = np.arange(1,17)
df_avg = pd.merge(df_avg,
         pd.melt(df_grocery_original.reset_index(),id_vars='index',var_name = 'hour',value_name='hourly_sales_original').sort_values(by=['index','hour']).reset_index(drop=True),
         how="left",on=['index','hour'])
df_avg.head(55)

,index,hour,hourly_sales,censor,hourly_sales_avg,hourly_sales_original
0,1,1,2.0,False,1.650000,2.0
1,1,2,4.0,False,3.450000,4.0
2,1,3,4.0,False,3.350000,4.0
3,1,4,2.0,False,2.100000,2.0
4,1,5,4.0,False,2.850000,4.0
5,1,6,2.0,False,2.550000,2.0
6,1,7,4.0,False,3.000000,4.0
7,1,8,1.0,False,1.631579,1.0
8,1,9,5.0,False,2.631579,5.0
9,1,10,4.0,False,2.437500,4.0


In [37]:
df_avg['hourly_sales'] = df_avg.apply(lambda row: max(row['hourly_sales_avg'], row['hourly_sales_original'])
                                      if pd.isna(row['hourly_sales']) 
                                      else row['hourly_sales'], axis=1)
df_avg

,index,hour,hourly_sales,censor,hourly_sales_avg,hourly_sales_original
0,1,1,2.0,False,1.650000,2.0
1,1,2,4.0,False,3.450000,4.0
2,1,3,4.0,False,3.350000,4.0
3,1,4,2.0,False,2.100000,2.0
4,1,5,4.0,False,2.850000,4.0
...,...,...,...,...,...,...
315,20,12,3.0,False,2.928571,3.0
316,20,13,3.0,False,2.785714,3.0
317,20,14,2.0,False,2.363636,2.0
318,20,15,2.0,False,2.000000,2.0


In [38]:
# Step 3: Repeat alogrithmic steps until you have reached final demand
# update booking by accumulating the updated daily booking
df_avg['hourly_sales'] = df_avg.groupby('index').hourly_sales.cumsum()
# post results to df_booking 
df_grocery['avg'] = df_avg.groupby('index').hourly_sales.tail(1).values.round()
df_grocery

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,avg
1,2,4,4,2,4,2,4,1.0,5.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN,48.0
2,3,4,4,5,4,5,4,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.0
3,1,3,2,2,3,6,4,2.0,3.0,3.0,3.0,4.0,2.0,NaN,NaN,NaN,43.0
4,2,3,4,1,4,3,3,1.0,3.0,3.0,4.0,2.0,3.0,2.0,NaN,NaN,41.0
5,2,5,5,3,5,5,5,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.0
6,1,3,3,1,3,3,2,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,31.0
7,3,4,3,2,4,5,5,3.0,2.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN,49.0
8,1,2,2,1,1,1,2,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,20.0
9,1,2,3,2,2,2,2,1.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,1.0,31.0
10,2,6,6,3,4,2,4,2.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.0


#### Comparing results of AVERAGING and DES method, averaging method is "less aggressive" than the DES method

### Part II: Demand Unconstraining: Service products with limited capacity

In [113]:
import pandas as pd
df_booking = pd.read_excel("booking_data.xlsx")
#df_booking.info()
df_booking

,Thursday,Book: day,Book: day.1,Book: day.2,Book: day.3,Book: day.4,Book: day.5,Book: day.6,Book: day.7,Book: day.8,...,Book: day.30,Book: day.31,Book: day.32,Book: day.33,Book: day.34,Book: day.35,Book: day.36,Book: day.37,Book: day.38,Book: day.39
0,Night #,40,39,38,37,36,35,34,33,32,...,10,9,8,7,6,5,4,3,2,1
1,1,0,0,0,1,1,1,1,1,2,...,16,16,18,18,18,19,20,20,21,23
2,2,0,1,1,1,1,1,1,1,1,...,25,25,25,25,25,25,25,25,25,25
3,3,1,1,2,2,2,2,2,2,3,...,22,25,25,25,25,25,25,25,25,25
4,4,0,0,1,1,1,1,1,1,1,...,11,11,13,14,14,14,14,15,15,16
5,5,0,1,3,4,6,7,9,9,11,...,24,24,25,25,25,25,25,25,25,25
6,6,0,2,4,5,5,5,5,7,7,...,21,21,23,24,25,25,25,25,25,25
7,7,0,0,1,2,2,3,3,3,4,...,18,18,19,21,23,23,23,24,24,25
8,8,0,0,0,0,0,0,0,0,1,...,13,14,15,15,17,17,18,19,19,19
9,9,0,0,0,0,1,2,3,4,5,...,13,13,15,15,16,16,16,18,20,20


In [114]:
# remove first row and first column 
# now each row is a booking curve
df_booking = df_booking.iloc[1:,1:]
df_booking.head(20)

,Book: day,Book: day.1,Book: day.2,Book: day.3,Book: day.4,Book: day.5,Book: day.6,Book: day.7,Book: day.8,Book: day.9,...,Book: day.30,Book: day.31,Book: day.32,Book: day.33,Book: day.34,Book: day.35,Book: day.36,Book: day.37,Book: day.38,Book: day.39
1,0,0,0,1,1,1,1,1,2,2,...,16,16,18,18,18,19,20,20,21,23
2,0,1,1,1,1,1,1,1,1,3,...,25,25,25,25,25,25,25,25,25,25
3,1,1,2,2,2,2,2,2,3,3,...,22,25,25,25,25,25,25,25,25,25
4,0,0,1,1,1,1,1,1,1,2,...,11,11,13,14,14,14,14,15,15,16
5,0,1,3,4,6,7,9,9,11,12,...,24,24,25,25,25,25,25,25,25,25
6,0,2,4,5,5,5,5,7,7,7,...,21,21,23,24,25,25,25,25,25,25
7,0,0,1,2,2,3,3,3,4,5,...,18,18,19,21,23,23,23,24,24,25
8,0,0,0,0,0,0,0,0,1,3,...,13,14,15,15,17,17,18,19,19,19
9,0,0,0,0,1,2,3,4,5,5,...,13,13,15,15,16,16,16,18,20,20
10,0,2,3,3,3,3,3,3,3,3,...,17,17,17,17,17,21,22,23,24,25


In [115]:
#Rename the column
import numpy as np
df_booking.columns = np.arange(1,41)
df_booking.head(20)

,1,2,3,4,5,6,7,8,9,10,...,31,32,33,34,35,36,37,38,39,40
1,0,0,0,1,1,1,1,1,2,2,...,16,16,18,18,18,19,20,20,21,23
2,0,1,1,1,1,1,1,1,1,3,...,25,25,25,25,25,25,25,25,25,25
3,1,1,2,2,2,2,2,2,3,3,...,22,25,25,25,25,25,25,25,25,25
4,0,0,1,1,1,1,1,1,1,2,...,11,11,13,14,14,14,14,15,15,16
5,0,1,3,4,6,7,9,9,11,12,...,24,24,25,25,25,25,25,25,25,25
6,0,2,4,5,5,5,5,7,7,7,...,21,21,23,24,25,25,25,25,25,25
7,0,0,1,2,2,3,3,3,4,5,...,18,18,19,21,23,23,23,24,24,25
8,0,0,0,0,0,0,0,0,1,3,...,13,14,15,15,17,17,18,19,19,19
9,0,0,0,0,1,2,3,4,5,5,...,13,13,15,15,16,16,16,18,20,20
10,0,2,3,3,3,3,3,3,3,3,...,17,17,17,17,17,21,22,23,24,25


In [117]:
# adjust the grocery data of sales by removing the sales possibly affected by out-of-stock
# check at which hour the demand becomes censored
# note that most summarize methods could be applied row-wise with axis = 1 #(25!!!!)
df_count= df_booking.count(axis=1)

for i in range(1,21):
    if df_booking.isnull().any(axis = 1).loc[i]: 
        df_booking.loc[i,df_count[i]+1] = 25

df_count= df_booking.count(axis=1)

In [116]:
# Step 1: Tease out the constrained booking curves
df_booking = df_booking.replace(25,np.nan)
df_booking

,1,2,3,4,5,6,7,8,9,10,...,31,32,33,34,35,36,37,38,39,40
1,0,0,0,1,1,1,1,1,2,2,...,16.0,16.0,18.0,18.0,18.0,19.0,20.0,20.0,21.0,23.0
2,0,1,1,1,1,1,1,1,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,2,2,2,2,2,2,3,3,...,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,1,1,1,1,1,1,1,2,...,11.0,11.0,13.0,14.0,14.0,14.0,14.0,15.0,15.0,16.0
5,0,1,3,4,6,7,9,9,11,12,...,24.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,2,4,5,5,5,5,7,7,7,...,21.0,21.0,23.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN
7,0,0,1,2,2,3,3,3,4,5,...,18.0,18.0,19.0,21.0,23.0,23.0,23.0,24.0,24.0,NaN
8,0,0,0,0,0,0,0,0,1,3,...,13.0,14.0,15.0,15.0,17.0,17.0,18.0,19.0,19.0,19.0
9,0,0,0,0,1,2,3,4,5,5,...,13.0,13.0,15.0,15.0,16.0,16.0,16.0,18.0,20.0,20.0
10,0,2,3,3,3,3,3,3,3,3,...,17.0,17.0,17.0,17.0,17.0,21.0,22.0,23.0,24.0,NaN


In [118]:
#step2: Create a daily demand dataframe

booking_demand = df_booking.columns[1:]
# Calculate the daily booking from cumulative booking
df_daily_demand = df_booking.copy()
df_daily_demand[booking_demand] = df_booking[booking_demand].diff(axis=1)
df_daily_demand[booking_demand] = df_daily_demand[booking_demand].fillna(df_booking[booking_demand])
# Return the modified dataframe
df_daily_demand.iloc[:, 1] = df_daily_demand.iloc[:, 1].astype(int)

df_daily_demand

,1,2,3,4,5,6,7,8,9,10,...,31,32,33,34,35,36,37,38,39,40
1,0,0,0,1,0,0,0,0,1,0,...,0.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,1.0,2.0
2,0,1,0,0,0,0,0,0,0,2,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,1,0,0,0,0,0,1,0,...,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,1,0,0,0,0,0,0,1,...,0.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
5,0,1,2,1,2,1,2,0,2,1,...,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,2,2,1,0,0,0,2,0,0,...,0.0,0.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
7,0,0,1,1,0,1,0,0,1,1,...,0.0,0.0,1.0,2.0,2.0,0.0,0.0,1.0,0.0,1.0
8,0,0,0,0,0,0,0,0,1,2,...,1.0,1.0,1.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0
9,0,0,0,0,1,1,1,1,1,0,...,0.0,0.0,2.0,0.0,1.0,0.0,0.0,2.0,2.0,0.0
10,0,2,1,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,1.0,1.0


In [119]:
df_booking = df_booking.astype(float)
df_daily_demand = df_daily_demand.astype(float)

In [120]:
#STEP 2: calculate the daily demand to cumulative demand ratio (r)
df_ratios = round(df_daily_demand / df_booking, 3)
df_ratios.head()

,1,2,3,4,5,6,7,8,9,10,...,31,32,33,34,35,36,37,38,39,40
1,NaN,NaN,NaN,1.00,0.000,0.000,0.000,0.0,0.500,0.000,...,0.000,0.00,0.111,0.000,0.0,0.053,0.05,0.000,0.048,0.087
2,NaN,1.0,0.000,0.00,0.000,0.000,0.000,0.0,0.000,0.667,...,0.040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,0.500,0.00,0.000,0.000,0.000,0.0,0.333,0.000,...,0.091,0.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,1.000,0.00,0.000,0.000,0.000,0.0,0.000,0.500,...,0.000,0.00,0.154,0.071,0.0,0.000,0.00,0.067,0.000,0.062
5,NaN,1.0,0.667,0.25,0.333,0.143,0.222,0.0,0.182,0.083,...,0.042,0.00,0.040,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
# STEP 3: calculate the average ratios for each day
df_average_ratios = np.nanmean(df_ratios, axis=0).round(3)
print(df_average_ratios)
# df_booking.info()
# STEP 4: calculate R (cumulative/total) and Q (hourly/total)
# applying initial values for R_m and Q_m before recursive calculations
# R_m = 1
# Q_m = r_m, where m is the last daily interval of the day
array_R = np.ones(len(df_average_ratios))
array_Q = np.ones(len(df_average_ratios))
# Q_40 = r_40
array_Q[39] = df_average_ratios[39]
# calculate the R and Q backwardly
# remember that numpy array index starts from 0. so index = 15 is for hour = 16
# applying formulas to fill out remaining values
# R_i-1 = R_i - Q_i
# Q_i-1 = r_i-1 * R_i-1
for j in range(len(df_average_ratios)-2,-1,-1): 
    array_R[j] = array_R[j+1] - array_Q[j+1]
    array_Q[j] = df_average_ratios[j]*array_R[j]

[1.    1.    0.444 0.302 0.158 0.134 0.085 0.127 0.248 0.157 0.148 0.134
 0.11  0.071 0.066 0.071 0.078 0.058 0.082 0.037 0.036 0.071 0.072 0.066
 0.037 0.033 0.055 0.044 0.061 0.031 0.033 0.028 0.061 0.03  0.044 0.04
 0.035 0.042 0.022 0.036]


In [122]:
# STEP 5: summarize total estimated demand for all booking curves as a column onto 
#the original dataset (unconstrained total demand = last unconstrainted cumulative demand / R ratio)
df_count[df_count != 40] = df_count[df_count != 40] -1
df_booking['position'] = df_count
# ensure position location is as expected
df_booking

,1,2,3,4,5,6,7,8,9,10,...,32,33,34,35,36,37,38,39,40,position
1,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,...,16.0,18.0,18.0,18.0,19.0,20.0,20.0,21.0,23.0,40
2,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30
3,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,...,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31
4,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,11.0,13.0,14.0,14.0,14.0,14.0,15.0,15.0,16.0,40
5,0.0,1.0,3.0,4.0,6.0,7.0,9.0,9.0,11.0,12.0,...,24.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32
6,0.0,2.0,4.0,5.0,5.0,5.0,5.0,7.0,7.0,7.0,...,21.0,23.0,24.0,25.0,NaN,NaN,NaN,NaN,NaN,34
7,0.0,0.0,1.0,2.0,2.0,3.0,3.0,3.0,4.0,5.0,...,18.0,19.0,21.0,23.0,23.0,23.0,24.0,24.0,25.0,40
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,...,14.0,15.0,15.0,17.0,17.0,18.0,19.0,19.0,19.0,40
9,0.0,0.0,0.0,0.0,1.0,2.0,3.0,4.0,5.0,5.0,...,13.0,15.0,15.0,16.0,16.0,16.0,18.0,20.0,20.0,40
10,0.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,17.0,17.0,17.0,17.0,21.0,22.0,23.0,24.0,25.0,40


In [123]:
# extract last cumulative demand using the diagonal trick
df_booking['demand_cum'] = np.diag(df_booking.iloc[:,df_booking.position-1])
# R ratio for the position
df_booking['R'] = array_R[df_booking.position-1]
# calculate and store unconstrained total demand by proportional method
df_booking['prop'] = (df_booking['demand_cum']/df_booking['R']).round()
df_booking

,1,2,3,4,5,6,7,8,9,10,...,35,36,37,38,39,40,position,demand_cum,R,prop
1,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,...,18.0,19.0,20.0,20.0,21.0,23.0,40,23.0,1.000000,23.0
2,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,30,24.0,0.684807,35.0
3,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,31,22.0,0.708176,31.0
4,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,14.0,14.0,14.0,15.0,15.0,16.0,40,16.0,1.000000,16.0
5,0.0,1.0,3.0,4.0,6.0,7.0,9.0,9.0,11.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,32,24.0,0.728577,33.0
6,0.0,2.0,4.0,5.0,5.0,5.0,5.0,7.0,7.0,7.0,...,25.0,NaN,NaN,NaN,NaN,NaN,34,24.0,0.799904,30.0
7,0.0,0.0,1.0,2.0,2.0,3.0,3.0,3.0,4.0,5.0,...,23.0,23.0,23.0,24.0,24.0,25.0,40,25.0,1.000000,25.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,...,17.0,17.0,18.0,19.0,19.0,19.0,40,19.0,1.000000,19.0
9,0.0,0.0,0.0,0.0,1.0,2.0,3.0,4.0,5.0,5.0,...,16.0,16.0,16.0,18.0,20.0,20.0,40,20.0,1.000000,20.0
10,0.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,17.0,21.0,22.0,23.0,24.0,25.0,40,25.0,1.000000,25.0
